In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

In [2]:
from utils import *
from datetime import date

In [3]:
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


### Batchnorm Training
- `model_bn` is the model with batchnorm layers
- train this model on the CIFAR 10 dataset

In [4]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.Tanh(),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.Tanh(),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.BatchNorm1d(1152),
    nn.Tanh(),
    nn.Linear(1152, 512),
    nn.BatchNorm1d(512),
    nn.Tanh(),
    nn.Linear(512, 128),
    nn.BatchNorm1d(128),
    nn.Tanh(),
    nn.Linear(128, 10)
)

In [5]:
loss_history_bn = []
writer_bn = SummaryWriter(log_dir='runs/bn_stats-%s' % date.today(), flush_secs=10)

In [6]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')
def criterion(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) #+ (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))


In [7]:
loss_tanh = train_model(model_bn, trainloader, criterion, optimizer = optim.Adam(list(model_bn.parameters()), lr=3e-4), epochs=20, writer=writer_bn)

[1,   300] loss: 74.074
[1,   600] loss: 86.137
[2,   300] loss: 71.429
[2,   600] loss: 56.982
[3,   300] loss: 64.992
[3,   600] loss: 61.539
[4,   300] loss: 57.573
[4,   600] loss: 60.354
[5,   300] loss: 59.456
[5,   600] loss: 45.987
[6,   300] loss: 49.959
[6,   600] loss: 65.092
[7,   300] loss: 48.579
[7,   600] loss: 57.464
[8,   300] loss: 45.842
[8,   600] loss: 39.021
[9,   300] loss: 36.963
[9,   600] loss: 36.688
[10,   300] loss: 34.752
[10,   600] loss: 30.771
[11,   300] loss: 36.247
[11,   600] loss: 19.673
[12,   300] loss: 30.540
[12,   600] loss: 16.395
[13,   300] loss: 25.840
[13,   600] loss: 24.904
[14,   300] loss: 26.002
[14,   600] loss: 29.593
[15,   300] loss: 19.097
[15,   600] loss: 12.667
[16,   300] loss: 12.360
[16,   600] loss: 17.417
[17,   300] loss: 10.826
[17,   600] loss: 11.987
[18,   300] loss: 7.680
[18,   600] loss: 5.659
[19,   300] loss: 14.996
[19,   600] loss: 7.228
[20,   300] loss: 8.215
[20,   600] loss: 13.650


In [10]:
stats_bn = print_stats(model_bn)

Mean:  0.030711716
Standard Deviation:  0.032555684
Sparsity:  0.002111457954250536


In [11]:
bn_model_err = get_model_error(model_bn, testloader)

test error: 0.3059
